In [4]:
import re
import sys
from pyspark import SparkConf, SparkContext
conf = SparkConf()
sc = SparkContext(conf=conf)
lines = sc.textFile('soc-short.txt')
user_friend = lines.map(lambda l:tuple(l.split('\t')))
user_ratingprod = user_friend.flatMap(lambda x:[((x[0],i),-1000) for i in x[1].split(',')])
user_ratingprod2 = user_friend.flatMap(lambda x: (lambda j:[[((j[u],j[i]),1) for i in range(u+1,len(j))]for u in range(0,len(j))])(x[1].split(',')))
user_ratingprod2 = user_ratingprod2.flatMap(lambda xs: [x for x in xs])
unio = user_ratingprod.union(user_ratingprod2).reduceByKey(lambda n1,n2:n1+n2)
unio = unio.map(lambda x: (x[0][0],x[0],x[1]))
key = unio.map(lambda x: (x[0]))
lst = key.collect()
uni_list = []
for i in lst:
    if i not in uni_list:
        uni_list.append(i)
charRDD = []
for i in uni_list:
    char = i
    user = unio.filter(lambda x: x[0]==char)
    top_10 = user.filter(lambda u:u[2] >= 0).takeOrdered(10, key=lambda x: -x[2])
    if len(top_10) == 10:
        charRDD.append(top_10)
    else:
        user_friend = user.map(lambda x:x[1][1]).collect()
        
        for j in uni_list:
            if j not in user_friend:
                top_10.append((char,(char,j),"random friend"))
                if len(top_10) == 10:
                    charRDD.append(top_10)
                    break
charRDD = sc.parallelize(charRDD)
#charRDD.saveAsTextFile(sys.argv[2])
#sc.stop()
charRDD.collect()

[[('0', ('0', '38737'), 3),
  ('0', ('0', '6027'), 2),
  ('0', ('0', '135'), 2),
  ('0', ('0', '22939'), 2),
  ('0', ('0', '35633'), 2),
  ('0', ('0', '44846'), 2),
  ('0', ('0', '30257'), 2),
  ('0', ('0', '32828'), 2),
  ('0', ('0', '1532'), 2),
  ('0', ('0', '27679'), 2)],
 [('1', ('1', '4'), 1),
  ('1', ('1', '8'), 1),
  ('1', ('1', '9'), 1),
  ('1', ('1', '10'), 1),
  ('1', ('1', '12'), 1),
  ('1', ('1', '14'), 1),
  ('1', ('1', '16'), 1),
  ('1', ('1', '17'), 1),
  ('1', ('1', '19'), 1),
  ('1', ('1', '21'), 1)],
 [('2', ('2', '3'), 1),
  ('2', ('2', '5'), 1),
  ('2', ('2', '6'), 1),
  ('2', ('2', '7'), 1),
  ('2', ('2', '11'), 1),
  ('2', ('2', '13'), 1),
  ('2', ('2', '15'), 1),
  ('2', ('2', '18'), 1),
  ('2', ('2', '23'), 1),
  ('2', ('2', '25'), 1)],
 [('3', ('3', '30'), 2),
  ('3', ('3', '38'), 2),
  ('3', ('3', '85'), 2),
  ('3', ('3', '89'), 2),
  ('3', ('3', '10'), 2),
  ('3', ('3', '16'), 2),
  ('3', ('3', '29'), 2),
  ('3', ('3', '83'), 2),
  ('3', ('3', '5'), 1),
  ('